In [1]:
import requests
import aiohttp
from bs4 import BeautifulSoup
import asyncio
import requests

In [2]:
url = f'https://newsapi.org/v2/everything?q=Netflix&sortBy=popularity&apiKey=8ebd90bb204348509808d65a70553855'
response = requests.get(url)

In [3]:
temp=response.json()


In [4]:
articles = temp.get("articles", [])
top_10_news = articles[:10]



In [5]:
print(top_10_news)

[{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'Matt Kamen', 'title': 'Philips GamePix 900 Review: Good Looks, Insane Value', 'description': 'This projector has remarkable picture quality, but few features beyond gaming performance.', 'url': 'https://www.wired.com/review/philips-gamepix-900/', 'urlToImage': 'https://media.wired.com/photos/67c74542d430c8b68c68bc0f/191:100/w_1280,c_limit/Philips-GamePix-900-Projector-Reviewer-Collage-032025-SOURCE-Matt-Kamen.jpg', 'publishedAt': '2025-03-05T13:33:00Z', 'content': 'The home projector field is competitive, so a new contender offering a 120" display, 4K resolution with HDR support, and lighting-fast refresh rates makes for an alluring prospect. That Philips is pr… [+2555 chars]'}, {'source': {'id': 'the-verge', 'name': 'The Verge'}, 'author': 'Ash Parrish', 'title': 'Netflix Games is getting its first MMO', 'description': 'Netflix just announced Spirit Crossing during a press event at GDC 2025. Spirit Crossing represents a first fo

In [6]:
dataset = {
    "Company": "Netflix",
    "Articles": [
        {
            "Title": article.get("title", "No headline available"),
            "description": article.get("description", "No summary available"),
            "url":article.get("url","No Url Available"),
            "content": article.get("content", "")

        }
        for article in top_10_news
    ]
}


In [7]:
import json
var=json.dumps(dataset, indent=4)
print(var)

{
    "Company": "Netflix",
    "Articles": [
        {
            "Title": "Philips GamePix 900 Review: Good Looks, Insane Value",
            "description": "This projector has remarkable picture quality, but few features beyond gaming performance.",
            "url": "https://www.wired.com/review/philips-gamepix-900/",
            "content": "The home projector field is competitive, so a new contender offering a 120\" display, 4K resolution with HDR support, and lighting-fast refresh rates makes for an alluring prospect. That Philips is pr\u2026 [+2555 chars]"
        },
        {
            "Title": "Netflix Games is getting its first MMO",
            "description": "Netflix just announced Spirit Crossing during a press event at GDC 2025. Spirit Crossing represents a first for Netflix, becoming the service\u2019s first MMO offering. It was developed by Spry Fox, one of Netflix\u2019s in-house game studios, and will launch sometime \u2026",
            "url": "https://www.thever

In [8]:
def extract_webpage_content(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Improved content extraction for Wired.com
        main_content = soup.find('div', class_='article__content') or soup.find('main') or soup.find('article')  # Wired uses 'article__content'

        if main_content:
            content_text = main_content.get_text(strip=True)
        else:
            content_text = 'Could not extract main content'
        return {
            'content': content_text
        }

    except requests.RequestException as e:
        return f"Error fetching the URL: {e}"
    except Exception as e:
        return f"Error processing URL {url}: {e}"


for article in dataset["Articles"]:
    finnhub_url = article.get("url", "")
    print(f"Original URL: {finnhub_url}")
    if finnhub_url:
        content = extract_webpage_content(finnhub_url)
        article["article_content"] = content 
    else:
        article["article_content"] = "No URL provided"

var=json.dumps(dataset, indent=4)
print(var)


Original URL: https://www.wired.com/review/philips-gamepix-900/
Original URL: https://www.theverge.com/news/632526/spirit-crossing-netflix-games-mmo-spry-fox
Original URL: https://www.theverge.com/news/625236/netflix-live-boxing-match-amanda-serrano-katie-taylor
Original URL: https://www.theverge.com/news/632264/netflix-carl-rinsch-white-horse-fraud-charges
Original URL: https://www.theverge.com/news/635280/netflix-hdr10-plus-support-samsung-tvs
Original URL: https://www.theverge.com/streaming-wars/630997/youtube-dude-perfect-mark-rober-youtubers-netflix-amazon-deals
Original URL: https://www.theverge.com/news/626102/netflix-18-billion-content-spending
Original URL: https://www.theverge.com/news/623171/youtube-tv-app-redesign-sell-streaming-services
Original URL: https://www.theverge.com/trailers/627307/devil-may-cry-netflix-vergil
Original URL: https://www.theverge.com/news/629242/black-mirror-season-7-trailer-date-netflix
{
    "Company": "Netflix",
    "Articles": [
        {
      

In [9]:
print(var)

{
    "Company": "Netflix",
    "Articles": [
        {
            "Title": "Philips GamePix 900 Review: Good Looks, Insane Value",
            "description": "This projector has remarkable picture quality, but few features beyond gaming performance.",
            "url": "https://www.wired.com/review/philips-gamepix-900/",
            "content": "The home projector field is competitive, so a new contender offering a 120\" display, 4K resolution with HDR support, and lighting-fast refresh rates makes for an alluring prospect. That Philips is pr\u2026 [+2555 chars]",
            "article_content": {
                "content": "Matt KamenGearMar 5, 2025 8:33 AMReview: Philips GamePix 900 ProjectorThis projector has remarkable picture quality, but few features beyond gaming performance.Photograph: Matt Kamen; Getty ImagesSave this storySaveSave this storySave$599at PhilipsAll products featured on WIRED are independently selected by our editors. However, when you buy something through our 

In [10]:
import os
import json
from dotenv import load_dotenv
from groq import Groq
import re

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=1024,
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

In [11]:
agent = Agent(model_name="llama3-8b-8192")


In [12]:
import os
import json
import re
from dotenv import load_dotenv
from groq import Groq
from collections import Counter

load_dotenv()

class Agent:
    def __init__(self, model_name):
        self.client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        self.model_name = model_name

    def run(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=self.model_name,
            temperature=0.2,
            max_tokens=512,
            top_p=0.9
        )
        return chat_completion.choices[0].message.content

def truncate_text(text, max_tokens=500):
    words = text.split()
    return ' '.join(words[:max_tokens])

def get_sentiment_from_groq(text):
    text = truncate_text(text)
    
    prompt = f""" 
    Analyze the sentiment of this news excerpt. 
    Respond with ONLY "Positive", "Negative", or "Neutral".

    Excerpt: 
    {text} 

    Sentiment: 
    """

    response = agent.run(prompt)

    sentiment = response.strip().lower()

    if "positive" in sentiment:
        return "Positive"
    elif "negative" in sentiment:
        return "Negative"
    else:
        return "Neutral"

def extract_topics_from_groq(text):
    text = truncate_text(text)
    
    prompt = f""" 
    Extract 3-5 precise, distinct topics from this news excerpt. 
    Respond ONLY with comma-separated topic names. 
    Avoid generic terms. Be specific and meaningful.

    Excerpt: 
    {text} 

    Specific Topics:
    """

    response = agent.run(prompt)

    def clean_topic(topic):
        # Remove any lingering instructions or extra text
        topic = re.sub(r'^(Here are the topics:|Topics:|3-5 key topics:)', '', topic, flags=re.IGNORECASE)
        topic = re.sub(r'^["\']|["\']$', '', topic.strip())
        return topic if topic else None

    # Split, clean, and filter topics
    topics = [clean_topic(topic) for topic in response.split(',')]
    topics = [topic for topic in topics if topic]
    
    return topics[:5]

def generate_coverage_differences(articles):
    # Ensure we have at least two articles
    if len(articles) < 2:
        return [{"Comparison": "Insufficient articles for comparative analysis.", "Impact": "No comparative insights available."}]
    
    # Generate a detailed prompt for coverage differences
    coverage_prompt = f"""
    Analyze the following two articles about Tesla:

    Article 1:
    - Title: {articles[0].get('Title', 'Untitled')}
    - Topics: {articles[0].get('Topics', [])}
    - Sentiment: {articles[0].get('Sentiment', 'Unspecified')}
    - Summary: {articles[0].get('Summary', 'No summary available')}

    Article 2:
    - Title: {articles[1].get('Title', 'Untitled')}
    - Topics: {articles[1].get('Topics', [])}
    - Sentiment: {articles[1].get('Sentiment', 'Unspecified')}
    - Summary: {articles[1].get('Summary', 'No summary available')}

    Provide a detailed comparison that focuses on:
    1. Narrative differences between the two articles
    2. Contrasting perspectives
    3. Potential market or investor implications

    Format your response as:
    Comparison: [Specific narrative differences]
    Impact: [Market or investor implications]
    """

    try:
        coverage_response = agent.run(coverage_prompt)
        
        # Extract Comparison and Impact
        comparison_match = re.search(r'Comparison:\s*(.+?)(?=Impact:|$)', coverage_response, re.DOTALL)
        impact_match = re.search(r'Impact:\s*(.+)$', coverage_response, re.DOTALL)
        
        comparison = comparison_match.group(1).strip() if comparison_match else (
            "Articles present different perspectives on Tesla's recent developments."
        )
        impact = impact_match.group(1).strip() if impact_match else (
            "Varying coverage may create mixed signals for investors and market perception."
        )

        return [{
            "Comparison": comparison,
            "Impact": impact
        }]
    
    except Exception as e:
        # Fallback if analysis generation fails
        return [{
            "Comparison": "Diverse coverage highlighting different aspects of Tesla's business.",
            "Impact": "Multiple perspectives provide a nuanced view of the company's current situation."
        }]

# Modify the main script to use this function
def generate_comparative_analysis(articles):
    # Existing code for sentiment distribution and topic processing
    sentiment_distribution = Counter(article['Sentiment'] for article in articles)
    
    all_topics = [topic for article in articles for topic in article['Topics']]
    unique_topics = [set(article['Topics']) for article in articles]
    common_topics = set.intersection(*unique_topics) if unique_topics else []
    
    # Unique Topics per Article
    unique_topics_per_article = [
        set(article['Topics']) - set.union(*(unique_topics[:i] + unique_topics[i+1:]))
        for i, article in enumerate(articles)
    ]

    # Generate coverage differences
    coverage_differences = generate_coverage_differences(articles)
    
    final_sentiment_prompt = f"""
    Analyze Tesla news coverage:
    - Article Sentiments: {[article['Sentiment'] for article in articles]}
    - Common Themes: {list(common_topics)}

    Provide a concise overall sentiment summary focusing on potential market implications.
    """
    
    final_sentiment_analysis = agent.run(final_sentiment_prompt)

    # Construct Comparative Analysis
    comparative_analysis = {
        "Comparative Sentiment Score": {
            "Sentiment Distribution": dict(sentiment_distribution),
            "Coverage Differences": coverage_differences,
            "Topic Overlap": {
                "Common Topics": list(common_topics),
                "Unique Topics in Article 1": list(unique_topics_per_article[0]) if unique_topics_per_article else [],
                "Unique Topics in Article 2": list(unique_topics_per_article[1]) if len(unique_topics_per_article) > 1 else []
            }
        },
        "Final Sentiment Analysis":final_sentiment_analysis
    }
    
    return comparative_analysis

# Initialize the agent with the specific model
agent = Agent(model_name="llama3-8b-8192")

if isinstance(var, str):
    flag1 = json.loads(var)

for article in flag1["Articles"]:
    content_text = article["article_content"]["content"]
    
    sentiment = get_sentiment_from_groq(content_text)
    article["Sentiment"] = sentiment
    
    topics = extract_topics_from_groq(content_text)
    article["Topics"] = topics

flag1["Comparative Analysis"] = generate_comparative_analysis(flag1["Articles"])

print(json.dumps(flag1, indent=4))


{
    "Company": "Netflix",
    "Articles": [
        {
            "Title": "Philips GamePix 900 Review: Good Looks, Insane Value",
            "description": "This projector has remarkable picture quality, but few features beyond gaming performance.",
            "url": "https://www.wired.com/review/philips-gamepix-900/",
            "content": "The home projector field is competitive, so a new contender offering a 120\" display, 4K resolution with HDR support, and lighting-fast refresh rates makes for an alluring prospect. That Philips is pr\u2026 [+2555 chars]",
            "article_content": {
                "content": "Matt KamenGearMar 5, 2025 8:33 AMReview: Philips GamePix 900 ProjectorThis projector has remarkable picture quality, but few features beyond gaming performance.Photograph: Matt Kamen; Getty ImagesSave this storySaveSave this storySave$599at PhilipsAll products featured on WIRED are independently selected by our editors. However, when you buy something through our 

In [14]:
print(json.dumps(flag1, indent=4))

{
    "Company": "Netflix",
    "Articles": [
        {
            "Title": "Philips GamePix 900 Review: Good Looks, Insane Value",
            "description": "This projector has remarkable picture quality, but few features beyond gaming performance.",
            "url": "https://www.wired.com/review/philips-gamepix-900/",
            "content": "The home projector field is competitive, so a new contender offering a 120\" display, 4K resolution with HDR support, and lighting-fast refresh rates makes for an alluring prospect. That Philips is pr\u2026 [+2555 chars]",
            "article_content": {
                "content": "Matt KamenGearMar 5, 2025 8:33 AMReview: Philips GamePix 900 ProjectorThis projector has remarkable picture quality, but few features beyond gaming performance.Photograph: Matt Kamen; Getty ImagesSave this storySaveSave this storySave$599at PhilipsAll products featured on WIRED are independently selected by our editors. However, when you buy something through our 

In [15]:
import torch
import os
from transformers import pipeline
from gtts import gTTS
import soundfile as sf

a:\SentimentAnalysis\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import sentencepiece
print(sentencepiece.__version__)

0.2.0


In [37]:
import uuid
from gtts import gTTS
from deep_translator import GoogleTranslator

def translate_to_hindi_deep(text: str) -> str:
    """Translate input text to Hindi using deep-translator."""
    print("Translating text to Hindi using deep-translator...")
    try:
        hindi_text = GoogleTranslator(source='auto', target='hi').translate(text)
        print("Translation result:", hindi_text)
        return hindi_text
    except Exception as e:
        print(f"Translation error (deep-translator): {e}")
        return text

def generate_tts_deep(text: str, lang: str = 'hi') -> str:
    """Generate a TTS audio file for the given text (translated to Hindi) using deep-translator."""
    hindi_text = translate_to_hindi_deep(text)
    try:
        if not hindi_text or hindi_text.isascii():
            print("Warning: The translated text does not appear to be in Hindi. Received:", hindi_text)
        tts = gTTS(text=hindi_text, lang=lang)
        filename = f"tts_{uuid.uuid4().hex}.mp3"
        tts.save(filename)
        print(f"TTS generated and saved as: {filename}")
        return filename
    except Exception as e:
        print(f"Error generating TTS: {e}")
        return ""

# Example usage:
if __name__ == "__main__":
    sample_text = "Based on the article sentiments, the overall sentiment towards Tesla is overwhelmingly positive, with 9 out of 10 articles having a positive tone. This suggests that the majority of news coverage about Tesla is favorable and optimistic.\n\nFrom a market perspective, this could have several implications:\n\n1. **Increased investor confidence**: A high proportion of positive news coverage can boost investor confidence in the company, potentially leading to increased stock prices and trading volumes.\n2. **Improved brand reputation**: Positive news coverage can enhance Tesla's brand reputation, making it more attractive to customers and potentially increasing sales.\n3. **Increased demand for products**: Favorable news coverage can drive demand for Tesla's products, particularly electric vehicles, as more people become aware of the company's innovative technology and sustainable offerings.\n4. **Competitive advantage**: Tesla's positive news coverage can create a competitive advantage, as it sets the company apart from its rivals and makes it more attractive to customers and investors.\n\nHowever, it's essential to note that the absence of negative news coverage may indicate a lack of critical scrutiny, which can be beneficial for the company but also potentially lead to complacency. A balanced approach to news coverage, including both positive and negative articles, can provide a more nuanced understanding of the company's strengths and weaknesses.\n\nIn conclusion, the overwhelmingly positive sentiment towards Tesla in news coverage suggests a strong brand reputation, increased investor confidence, and potentially increased demand for its products. However, it's crucial to maintain a balanced perspective and consider both positive and negative news coverage to gain a comprehensive understanding of the company's performance."
    hindi_translation = translate_to_hindi_deep(sample_text)
    print("Hindi translation (deep):", hindi_translation)
    
    tts_filename = generate_tts_deep(sample_text)
    if tts_filename:
        print(f"Generated TTS audio file (deep): {tts_filename}")
    else:
        print("Failed to generate TTS audio using deep-translator.")


Translating text to Hindi using deep-translator...
Translation result: लेख की भावनाओं के आधार पर, टेस्ला के प्रति समग्र भावना अत्यधिक सकारात्मक है, जिसमें 10 में से 9 लेखों में एक सकारात्मक स्वर है। इससे पता चलता है कि टेस्ला के बारे में अधिकांश समाचार कवरेज अनुकूल और आशावादी है।

बाजार के नजरिए से, इसके कई निहितार्थ हो सकते हैं:

1। ** बढ़ा हुआ निवेशक विश्वास **: सकारात्मक समाचार कवरेज का एक उच्च अनुपात कंपनी में निवेशकों के विश्वास को बढ़ावा दे सकता है, जो संभावित रूप से स्टॉक की कीमतों और ट्रेडिंग वॉल्यूम में वृद्धि के लिए अग्रणी है।
2। ** बेहतर ब्रांड प्रतिष्ठा **: सकारात्मक समाचार कवरेज टेस्ला की ब्रांड प्रतिष्ठा को बढ़ा सकता है, जिससे यह ग्राहकों के लिए अधिक आकर्षक हो जाता है और संभावित रूप से बिक्री बढ़ जाती है।
3। ** उत्पादों की बढ़ी हुई मांग **: अनुकूल समाचार कवरेज टेस्ला के उत्पादों, विशेष रूप से इलेक्ट्रिक वाहनों की मांग को बढ़ा सकता है, क्योंकि अधिक लोग कंपनी की अभिनव प्रौद्योगिकी और टिकाऊ प्रसाद के बारे में जानते हैं।
4। ** प्रतिस्पर्धात्मक लाभ **: टेस्ला का पॉजिटिव न्यूज 